# Train a Machine Learning Model, Export to ONNX, and Register in Seeq

## This notebook walks through:

1. Training a simple Machine Learning model
2. Converting the trained model to ONNX format
3. Registering the ONNX in Seeq using `SeeqONNXClient`

## What is ONNX?
**[ONNX (Open Neural Network Exchange)](https://onnx.ai/)** is an open standard format for representing machine learning models.
- Interoperable across frameworks (e.g., PyTorch, Scikit-learn, TensorFlow)
- Ideal for model sharing, deployment, and runtime optimization

You can train a model in one framework and run it in another runtime that supports ONNX (e.g., ONNX Runtime).

## Example

In this example, we predict the Temperature of `Example >> Cooling Tower 1 >> Area A` using features like:

- Relative Humidity
- Optimizer
- Compressor Power
- Wet Bulb

We use the `RandomForestRegressor` model from `scikit-learn` for training. The trained model is then:

1. Exported to ONNX format
2. Registered in Seeq using the `SeeqONNXClient`

### Step 1: Import required modules

First, we import all the necessary Python libraries. These are needed to train the model, export it, and register it later. This step sets up everything we need for the process ahead.

In [1]:
# If you do not have required modules installed, install it using the following command:
!pip install scikit-learn onnx==1.17.0 skl2onnx

# Note: After installing , please restart the kernel to ensure the changes take effect.

In [2]:
from seeq import spy
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from source.seeq_onnx_helper import cast_double_to_float, describe_onnx
from source.seeq_onnx_helper import SeeqONNXClient

### Step 2: Load and Prepare Data

Next, we load the dataset and split it into training and testing sets. The training set will be used to train the model, while the testing set helps us check how well the model performs.

In [3]:
# This example uses the data from the Example >> Cooling Tower 1 >> Area A.

# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False, quiet=True)

results = spy.search({"Path": "Example >> Cooling Tower 1 >> Area A"}, quiet=True)
df = spy.pull(results, start='2025-01-01', end='2025-01-15', grid='30min', header='Name', quiet=True)

In [4]:
df.head()

,Relative Humidity,Optimizer,Temperature,Compressor Power,Wet Bulb,Compressor Stage
2025-01-01 00:00:00+00:00,44.222585,1.27119,93.483492,17.148297,75.742938,STAGE 1
2025-01-01 00:30:00+00:00,42.131650,1.95313,95.084983,17.398583,76.137111,STAGE 1
2025-01-01 01:00:00+00:00,43.378360,1.55407,95.210453,17.347173,76.754701,STAGE 1
2025-01-01 01:30:00+00:00,43.819000,1.95313,93.568286,17.390857,75.646889,STAGE 1
2025-01-01 02:00:00+00:00,43.379992,1.95313,95.052661,17.258744,76.631910,STAGE 1


In [5]:
train_columns = ["Relative Humidity", "Optimizer", "Compressor Power", "Wet Bulb"]
target_column = "Temperature"

X = df[train_columns]
y = df[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Step 3: Train a Model

Now we train a `RandomForestRegressor` using the training data. This is where the model learns the patterns and relationships needed to make predictions.

In [6]:
model = RandomForestRegressor(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

# Evaluate mean squared error
preds = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, preds))

MSE: 0.6234234528259502


### Step 4: Convert to ONNX

Once training is done, we convert the `RandomForestRegressor` model into ONNX format. This makes it ready for deployment across different platforms.

In [7]:
initial_type = [("input", FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the model
onnx_path = "temperature_predictor.onnx"
with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"ONNX model saved to {onnx_path}")

ONNX model saved to temperature_predictor.onnx


### Step 5: Register the ONNX with Seeq

Now that the model is exported to ONNX, we register it with the Seeq server. This step makes the model accessible from Seeq for use.


In [8]:
# Workbook to scope ONNX to

workbook_path = spy.DEFAULT_WORKBOOK_PATH
workbook_context = spy._context.WorkbookContext.from_string(session=spy.session, status=spy.Status(), workbook_arg=workbook_path, worksheet_arg=None, datasource_arg=None) 
workbook_id = workbook_context.workbook_object.id if workbook_context.workbook_object is not None else None

In [9]:
onnx_model = onnx.load(onnx_path)
describe_onnx(onnx_model)


=== Model: fc052c188a994ff59c9aa0eb831ffd58 ===

Inputs:
  - input: type=FLOAT, shape=['?', 4]

Outputs:
  - variable: type=FLOAT, shape=['?', 1]


#### Seeq requires inputs of type DOUBLE for precision, but the ONNX model exported above has FLOAT inputs.
##### we use the helper function `cast_double_to_float` to add a cast node to convert inputs from DOUBLE to FLOAT before proceeding.

In [10]:
casted_model = cast_double_to_float(onnx_model)
describe_onnx(casted_model)


=== Model: fc052c188a994ff59c9aa0eb831ffd58 ===

Inputs:
  - input: type=DOUBLE, shape=['?', 4]

Outputs:
  - variable: type=FLOAT, shape=['?', 1]


In [11]:
# Next, we initialize the SeeqONNXClient to connect with the Seeq server.

hostname = spy.client.host
token = spy.client.auth_token
seeq_onnx_client = SeeqONNXClient(hostname, token)

#### Now we will call the `register` function of `SeeqONNXClient` and register the ONNX in Seeq.
> **Note** Ensure that the labels list passed to register function maintains the same order as the features used during model training. Any mismatch in order can result in feeding incorrect signals into the model, leading to unreliable or inaccurate predictions.

In [12]:
onnx_id = seeq_onnx_client.register(
    name=f"Example temperature predictor {datetime.datetime.now().isoformat()}",
    description="A RandomForestRegressor trained to predict temperature of Example>>Cooling Tower 1>>Area A",
    model=casted_model,
    model_type="PREDICTION",
    labels=train_columns,
    scope=workbook_id
)

Successfully registered ONNX 'Example temperature predictor 2025-05-29T23:34:22.318295' (ID: 0F03CB75-94CC-7310-BA21-3822EEFDBE4D) (Scoped to: 0F03C58B-57A5-FFD0-A8BD-541B0268E890)


In [13]:
seeq_onnx_client.list()

,ID,Name,Description,Type,Input Labels,Feature Count,Scoped To,Archived
0,0F03CB75-94CC-7310-BA21-3822EEFDBE4D,Example temperature predictor 2025-05-29T23:34...,A RandomForestRegressor trained to predict tem...,PREDICTION,"[Relative Humidity, Optimizer, Compressor Powe...",4,0F03C58B-57A5-FFD0-A8BD-541B0268E890,False


### Done
Your Machine Learning model is trained, exported to ONNX, and registered on Seeq.

**Next steps:**
- Use the `External ML Tool` in the Tool Pane of Seeq
  - Select the registered ONNX from the dropdown
  - Map signals to ONNX inputs
  - Execute